# Active Buyer Rates

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
import datetime

In [2]:
bid=pd.read_excel('bid19to21.xlsx')
stock=pd.read_excel('stock19to21.xlsx')

In [3]:
bid.columns

Index(['Unnamed: 0', 'seller_username', 'seller_id', 'buyer_username',
       'buyer_id', 'item_id', 'title', 'brand_name', 'category_name',
       'overall_grade', 'min_bid', 'bid', 'winning_bid', 'bid_placed',
       'main_image', 'win'],
      dtype='object')

In [4]:
bid['buyer_username']=bid['buyer_username'].str.lower()
bid['win']='N'
bid.loc[bid['bid']>=bid['winning_bid'], 'win']='Y'
bid.head()

,Unnamed: 0,seller_username,seller_id,buyer_username,buyer_id,item_id,title,brand_name,category_name,overall_grade,min_bid,bid,winning_bid,bid_placed,main_image,win
0,0,aucnet,196448,quick consignment,34802,973197,M93448 Atsui MM,Louis Vuitton,Totes,B,1132.84,1555.0,1540.55,2019-01-06 09:50:23,https://img.leprix.com/image/huohp4xlM2aNy25hl...,Y
1,1,aucnet,196448,paul.s,186106,973213,TENDOVU PM,Louis Vuitton,Shoulder Bags,C,202.62,306.0,443.61,2019-01-04 12:52:04,https://img.leprix.com/image/kSmcIiogLJQpcO24k...,N
2,2,aucnet,196448,magdalena.t,196771,973229,8BR631 Chain shoulder Brown G fitting,Fendi,Shoulder Bags,B,92.10,256.0,256.00,2019-01-04 12:54:25,https://img.leprix.com/image/2c0rjIAJLXPkIJaqS...,Y
3,3,aucnet,196448,magdalena.t,196771,973232,189749 Shoulder bag Brown S bracket,Gucci,Shoulder Bags,B,165.78,266.0,422.92,2019-01-04 15:39:31,https://img.leprix.com/image/wP9Pw5OmpNB2TXv5r...,N
4,4,aucnet,196448,paul.s,186106,973285,Medium Quilted Chain Shoulder Black,Chanel,Shoulder Bags,C,994.69,1305.0,1302.54,2019-01-04 12:20:33,https://img.leprix.com/image/VU44cxW1rCKoH4v2K...,Y


In [5]:
stock.columns

Index(['seller_username', 'seller_id', 'buyer_username', 'buyer_id', 'item_id',
       'title', 'brand_name', 'category_name', 'overall_grade', 'price',
       'submitted_date', 'main_image'],
      dtype='object')

In [6]:
stock['buyer_username']=stock['buyer_username'].str.lower()

In [7]:
# stock.to_excel('stock19to21.xlsx')
# bid.to_excel('bid19to21.xlsx')

In [8]:
bid['bid_placed']=pd.to_datetime(bid['bid_placed'])
stock['submitted_date']=pd.to_datetime(stock['submitted_date'])

In [9]:
bid_act=pd.DataFrame(bid[bid['win']=='Y']['buyer_username'].unique())
stock_act=pd.DataFrame(stock['buyer_username'].unique())

active_all=bid_act.merge(stock_act, how='outer')
# active_all.to_excel('active_accts.xlsx')

In [10]:
may_b=bid[(bid['bid_placed'].dt.month.isin([3,4,5]))&(bid['bid_placed'].dt.year==2021)]
may_s=stock[(stock['submitted_date'].dt.month.isin([3,4,5]))&(stock['submitted_date'].dt.year==2021)]

In [11]:
bidders=pd.DataFrame(may_b['buyer_username'].unique())
stockers=pd.DataFrame(may_s['buyer_username'].unique())

In [12]:
active_may=bidders.merge(stockers, how='outer')

In [13]:
active_may.shape

(174, 1)

In [14]:
# set range of dates from may 2020-april 2021 (don't include may)
may_year_b=bid[(bid['bid_placed']<datetime.datetime(2021,5,1))&
               (bid['bid_placed']>datetime.datetime(2020,5,1))]
may_year_s=stock[(stock['submitted_date']<datetime.datetime(2021,5,1)) & 
                (stock['submitted_date']>datetime.datetime(2020,5,1))]

year_bid=pd.DataFrame(may_year_b['buyer_username'].unique())
year_stock=pd.DataFrame(may_year_s['buyer_username'].unique())

active_year=year_bid.merge(year_stock, how='outer')

In [15]:
active_may=active_may[active_may[0].isin(list(active_year[0]))]

In [16]:
active_may.shape

(167, 1)

In [17]:
print('3 Month Rolling Active Buyer Rate March-May 2021 ALL BIDS: {:.2f}%'.format(len(active_may)/len(active_year)*100))

3 Month Rolling Active Buyer Rate March-May 2021 ALL BIDS: 54.75%


In [18]:
def active_rate_3(bid, stock, month, year):
    """
    Prints 3 month rolling active buyer rate
    
    Parameter bid: the past bidding df
    Precondition: bid is a df
    
    Parameter stock: the past stock quotes df
    Precondition: stock is a df
    
    Parameter month: the end month to determine rate
    Precondition: month is an int between 1 and 12 
    
    Parameter year: the year to determine rate
    Precondition: year is an int formatted with century (i.e. 2021)
    """
    # only look at bidders who won and stock buyers
    bid=bid[(bid['win']=='Y')]       
    
    # set start month,year and end month,year
    if month<3:
        start_m=month-2+12
        start_yr=year-1
    else:
        start_m=month-2
        start_yr=year
    if month==12:
        end_m=1
        end_yr=year+1
    else:
        end_m=month+1
        end_yr=year
    
    # set date to 3 month period    
    b=bid[(bid['bid_placed']>datetime.datetime(start_yr, start_m, 1))&(bid['bid_placed']<datetime.datetime(end_yr,end_m,1))]
    s=stock[(stock['submitted_date']>datetime.datetime(start_yr, start_m, 1))&(stock['submitted_date']<datetime.datetime(end_yr,end_m,1))]
    
    # unique buyers from bid and stock over 3 month period
    bidders=pd.DataFrame(b['buyer_username'].unique())
    stockers=pd.DataFrame(s['buyer_username'].unique())
    
    active=bidders.merge(stockers, how='outer')
    
    # set date to rolling 12 months prior (month in the year prior up to but not including the current month)
    year_b=bid[(bid['bid_placed']<datetime.datetime(year,month,1))&
               (bid['bid_placed']>datetime.datetime(year-1,month,1))]
    year_s=stock[(stock['submitted_date']<datetime.datetime(year,month,1)) & 
                (stock['submitted_date']>datetime.datetime(year-1,month,1))]
    
    # unique buyers from bid and stock over rolling 12 months prior
    year_bid=pd.DataFrame(year_b['buyer_username'].unique())
    year_stock=pd.DataFrame(year_s['buyer_username'].unique())

    active_year=year_bid.merge(year_stock, how='outer')
    
    # only keep buyers from 3 month period who have bought in the past 12 months (in active_year)
    active=active[active[0].isin(list(active_year[0]))]
    
    print('3 Month Active Buyer Rate '+str(start_m)+'/'+str(start_yr)+'-'+str(month)+'/'+str(year)+': {:.2f}%'.format(len(active)/len(active_year)*100))

active_rate_3(bid,stock,2,2021)

3 Month Active Buyer Rate 12/2020-2/2021: 45.75%


In [19]:
def active_rate_monthly(bid, stock, month, year):
    """
    Prints monthly active buyer rate
    
    Parameter bid: the past bidding df
    Precondition: bid is a df
    
    Parameter stock: the past stock quotes df
    Precondition: stock is a df
    
    Parameter month: the month to determine rate
    Precondition: month is an int between 1 and 12
    
    Parameter year: the year to determine rate
    Precondition: year is an int formatted with century (i.e. 2021)
    """
    # only look at bidders who won and stock buyers
    bid=bid[(bid['win']=='Y')]
    
    # set date to one month period
    b=bid[(bid['bid_placed'].dt.month==month)&(bid['bid_placed'].dt.year==year)]
    s=stock[(stock['submitted_date'].dt.month==month)&(stock['submitted_date'].dt.year==year)]
    
    # unique buyers from bid and stock over one month period
    bidders=pd.DataFrame(b['buyer_username'].unique())
    stockers=pd.DataFrame(s['buyer_username'].unique())
    
    active=bidders.merge(stockers, how='outer')
    
    # set date to rolling 12 months prior (month in the year prior up to but not including the current month)
    year_b=bid[(bid['bid_placed']<datetime.datetime(year,month,1))&
               (bid['bid_placed']>datetime.datetime(year-1,month,1))]
    year_s=stock[(stock['submitted_date']<datetime.datetime(year,month,1)) & 
                (stock['submitted_date']>datetime.datetime(year-1,month,1))]
    
    # unique buyers from bid and stock over rolling 12 months prior
    year_bid=pd.DataFrame(year_b['buyer_username'].unique())
    year_stock=pd.DataFrame(year_s['buyer_username'].unique())

    active_year=year_bid.merge(year_stock, how='outer')
    
    # only keep buyers from month period who have bought in the past 12 months (in active_year)
    active=active[active[0].isin(list(active_year[0]))]
    
    print('Monthly Active Buyer Rate '+str(month)+'/'+str(year)+': {:.2f}%'.format(len(active)/len(active_year)*100))
    

In [20]:
for x in range(1,7):
    active_rate_3(bid,stock,x,2021)
    active_rate_monthly(bid,stock,x,2021)
    print('\n')

3 Month Active Buyer Rate 11/2020-1/2021: 45.42%
Monthly Active Buyer Rate 1/2021: 25.83%


3 Month Active Buyer Rate 12/2020-2/2021: 45.75%
Monthly Active Buyer Rate 2/2021: 27.13%


3 Month Active Buyer Rate 1/2021-3/2021: 49.60%
Monthly Active Buyer Rate 3/2021: 30.65%


3 Month Active Buyer Rate 2/2021-4/2021: 51.78%
Monthly Active Buyer Rate 4/2021: 30.83%


3 Month Active Buyer Rate 3/2021-5/2021: 53.91%
Monthly Active Buyer Rate 5/2021: 32.81%


3 Month Active Buyer Rate 4/2021-6/2021: 49.21%
Monthly Active Buyer Rate 6/2021: 30.31%




In [21]:
for x in range(1,13):
    active_rate_3(bid,stock,x,2020)
    active_rate_monthly(bid,stock,x,2020)
    print('\n')

3 Month Active Buyer Rate 11/2019-1/2020: 56.43%
Monthly Active Buyer Rate 1/2020: 32.86%


3 Month Active Buyer Rate 12/2019-2/2020: 54.67%
Monthly Active Buyer Rate 2/2020: 34.00%


3 Month Active Buyer Rate 1/2020-3/2020: 55.21%
Monthly Active Buyer Rate 3/2020: 25.15%


3 Month Active Buyer Rate 2/2020-4/2020: 55.43%
Monthly Active Buyer Rate 4/2020: 20.00%


3 Month Active Buyer Rate 3/2020-5/2020: 47.13%
Monthly Active Buyer Rate 5/2020: 24.71%


3 Month Active Buyer Rate 4/2020-6/2020: 45.45%
Monthly Active Buyer Rate 6/2020: 27.81%


3 Month Active Buyer Rate 5/2020-7/2020: 51.28%
Monthly Active Buyer Rate 7/2020: 33.33%


3 Month Active Buyer Rate 6/2020-8/2020: 51.98%
Monthly Active Buyer Rate 8/2020: 31.19%


3 Month Active Buyer Rate 7/2020-9/2020: 49.51%
Monthly Active Buyer Rate 9/2020: 28.64%


3 Month Active Buyer Rate 8/2020-10/2020: 49.32%
Monthly Active Buyer Rate 10/2020: 30.14%


3 Month Active Buyer Rate 9/2020-11/2020: 50.65%
Monthly Active Buyer Rate 11/2020: 24

### Churn

In [39]:
def churn(bid, stock, year):
    """
    Prints number of active buyers year prior and number of buyers in current year who dropped off
    
    Parameter bid: the past bidding df
    Precondition: bid is a df
    
    Parameter stock: the past stock quotes df
    Precondition: stock is a df
    
    Parameter year: the year to determine rate
    Precondition: year is an int formatted with century (i.e. 2021)
    """
    # only look at bidders who won and stock buyers
    bid=bid[(bid['win']=='Y')]       
    
    # set date to current year    
    b=bid[(bid['bid_placed'].dt.year==year)]
    s=stock[(stock['submitted_date'].dt.year==year)]
    
    # unique buyers from bid and stock for current year
    bidders=pd.DataFrame(b['buyer_username'].unique())
    stockers=pd.DataFrame(s['buyer_username'].unique())
    
    active=bidders.merge(stockers, how='outer')
    
    # set date to year prior
    year_b=bid[(bid['bid_placed'].dt.year==year-1)]
    year_s=stock[(stock['submitted_date'].dt.year==year-1)]
    
    # unique buyers from bid and stock from year prior
    year_bid=pd.DataFrame(year_b['buyer_username'].unique())
    year_stock=pd.DataFrame(year_s['buyer_username'].unique())

    active_year=year_bid.merge(year_stock, how='outer')
    
    # pull buyers who have not bought in the current year from year prior
    new_active=active[active[0].isin(list(active_year[0]))] # buyers from last year who also bought this year
    
    dropped=len(active_year)-len(new_active)
    
    print('Active Buyers in '+str(year-1)+': {:.0f}'.format(len(active_year)))
    print('Active Buyers in '+str(year-1)+' who did not buy in '+str(year)+': {:.0f}'.format(dropped))

churn(bid,stock,2021)

Active Buyers in 2020: 240
Active Buyers in 2020 who did not buy in 2021: 124
